In [41]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.contrib.fbbt.fbbt import fbbt
from pyomo.common.errors import InfeasibleConstraintException
import numpy as np
def solve(m, solver):
    
    opt = pyo.SolverFactory(solver)
    results = opt.solve(m)
    return m, results

In [42]:
# Question 2
def subproblem(y1, y2, y3):
    m = pyo.ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT)

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.y1 +1.5*m.y2 + 0.5*m.y3 + m.x1**2 + m.x2**2

    @m.Constraint()
    def g1(m):
        return (m.x1-2)**2 - m.x2 <= 0 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0
        
    @m.Constraint()
    def g12(m):
        return m.y1 == y1
        
    @m.Constraint()
    def g13(m):
        return m.y2 == y2
        
    @m.Constraint()
    def g14(m):
        return m.y3 == y3
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    return m


def subproblem_slack(y1, y2, y3):
    m = pyo.ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT)

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)
    m.u = pyo.Var(within=pyo.Reals)
    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.u

    @m.Constraint()
    def g1(m):
        return (m.x1-2)**2 - m.x2 <= m.u

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= m.u 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= m.u

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= m.u

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= m.u

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= m.u

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0
        
    @m.Constraint()
    def g12(m):
        return m.y1-y1==0
        
    @m.Constraint()
    def g13(m):
        return m.y2-y2==0
        
    @m.Constraint()
    def g14(m):
        return m.y3-y3==0
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.pprint()
    return m



In [43]:
# Generalized Benders Decomposition
def master_gbd():
    m = pyo.ConcreteModel()

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)
    m.con_base = Constraint(expr = 1 - m.y1 - m.y2 - m.y3 <= 0)
    m.alpha = pyo.Var(within=pyo.Reals)
    m.con_extra = ConstraintList()
    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha

    return m
UB =  float('inf') 
LB =  float('-inf') 
i = 1
y_k = [1,1,1]
m_master = master_gbd()
while(UB>LB):
    print('Iteration'+str(i))
    i = i+1
    m_sub = subproblem(y_k[0],y_k[1],y_k[2])
    ms, results = solve(m_sub, 'ipopt')
    print(results.solver.termination_condition)
    if(results.solver.termination_condition == TerminationCondition.optimal):
        print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
        print('Multipliers associated: | mu1:',ms.dual[ms.g12],'| mu2:',ms.dual[ms.g13],'| mu3:',ms.dual[ms.g14],'| feasible')
        xk1 = ms.x1()
        xk2 = ms.x2()
        mu1 = ms.dual[ms.g12]
        mu2 = ms.dual[ms.g13]
        mu3 = ms.dual[ms.g14]
        ub_temp = ms.y1() + 1.5*ms.y2() + 0.5*ms.y3() + xk1**2 + xk2**2
        m_master.con_extra.add(m_master.alpha >= m_master.y1 + 1.5*m_master.y2 + 0.5*m_master.y3 + xk1**2 + xk2**2 + mu1*(m_master.y1-y_k[0]) + mu2*(m_master.y2-y_k[1])+mu3*(m_master.y3-y_k[2]))
    else:
        m_sub = subproblem_slack(y_k[0],y_k[1],y_k[2])
        ms, results = solve(m_sub, 'ipopt')
        print(results.solver.termination_condition)
        print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
        print('Multipliers associated: | mu1:',ms.dual[ms.g12],'| mu2:',ms.dual[ms.g13],'| mu3:',ms.dual[ms.g14],'| infeasible')
        xk1 = ms.x1()
        xk2 = ms.x2()
        mu1 = ms.dual[ms.g12]
        mu2 = ms.dual[ms.g13]
        mu3 = ms.dual[ms.g14]
        #ub_temp1 = m_master.y1() + 1.5*m_master.y2() + 0.5*m_master.y3() + xk1**2 + xk2**2
        m_master.con_extra.add(0 >= mu1*(m_master.y1-y_k[0]) + mu2*(m_master.y2-y_k[1])+mu3*(m_master.y3-y_k[2]))
    m, results = solve(m_master, 'gurobi')
    y_k = [m.y1(),m.y2(),m.y3()]
    if ub_temp<=UB:
        UB = ub_temp
    LB = m.alpha()
    print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
    print('UB:', UB, 'and LB:', LB)
    print()
    
# Iteration 1

# ms.dual.pprint()


# Master




Iteration1
optimal
NLP: | x1: 1.999999990313079 | x2: 1.9999999509391835 | y1: 1.0 | y2: 1.0 | y3: 1.0 | Objective: 10.999999765009052 | Solver Status.: optimal
Multipliers associated: | mu1: 17.2505900974285 | mu2: 17.75058993867836 | mu3: 0.7505903612548934 | feasible
Master MILP: | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: -26.50118027109781 | Solver Status.: optimal
UB: 10.999999765009052 and LB: -26.50118027109781

Iteration2
optimal
NLP: | x1: 1.4999999963605903 | x2: 1.4999999944737907 | y1: 4.350279386575854e-40 | y2: 0.0 | y3: 1.0 | Objective: 4.999999972503144 | Solver Status.: optimal
Multipliers associated: | mu1: 0.498819290151971 | mu2: 0.998819296004342 | mu3: 9.499999952210047 | feasible
Master MILP: | y1: 1.0 | y2: 0.0 | y3: 0.0 | Objective: -3.501180689554932 | Solver Status.: optimal
UB: 4.999999972503144 and LB: -3.501180689554932

Iteration3
optimal
NLP: | x1: 1.9999999906264758 | x2: 0.00011128805529353067 | y1: 1.0 | y2: 2.6841541547693156e-43 | y3: 0.0 | Objectiv

In [37]:
#Outer approximation
def master_oa():
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    m.alpha = pyo.Var(within=pyo.Reals)
    m.con_extra = ConstraintList()

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha

    #@m.Constraint()
    #def alpha_con(m):
     #   return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2)


    #@m.Constraint()
    #def g1_lin(m):
     #   return (xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    return m

UB =  float('inf') 
LB =  float('-inf') 
i = 1
y_k = [1,1,1]
m_master = master_oa()
while(UB>LB):
    print('Iteration'+str(i))
    i = i+1
    m_sub = subproblem(y_k[0],y_k[1],y_k[2])
    ms, results = solve(m_sub, 'gurobi')
    print(results.solver.termination_condition)
    if(results.solver.termination_condition == TerminationCondition.optimal):
        print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
       
        xk1 = ms.x1()
        xk2 = ms.x2()
        
        ub_temp = ms.y1() + 1.5*ms.y2() + 0.5*ms.y3() + xk1**2 + xk2**2
        m_master.con_extra.add(m_master.y1 + 1.5*m_master.y2 + 0.5*m_master.y3 + xk1**2 + xk2**2 + 2*xk1*(m_master.x1 - xk1) + 2*xk2*(m_master.x2 - xk2)<=m_master.alpha)
        m_master.con_extra.add((xk1-2)**2-xk2+2*(xk1-2)*(m_master.x1-xk1)-(m_master.x2-xk2)<=0)
    else:
        m_sub = subproblem_slack(y_k[0],y_k[1],y_k[2])
        ms, results = solve(m_sub, 'gurobi')
        print(results.solver.termination_condition)
        print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
        
        xk1 = ms.x1()
        xk2 = ms.x2()
        mu1 = ms.dual[ms.g12]
        mu2 = ms.dual[ms.g13]
        mu3 = ms.dual[ms.g14]
        #ub_temp1 = m_master.y1() + 1.5*m_master.y2() + 0.5*m_master.y3() + xk1**2 + xk2**2
        m_master.con_extra.add(m_master.y1 + 1.5*m_master.y2 + 0.5*m_master.y3 + xk1**2 + xk2**2 + 2*xk1*(m_master.x1 - xk1) + 2*xk2*(m_master.x2 - xk2))
        m_master.con_extra.add((xk1-2)**2-xk2+2*(xk1-2)*(m_master.x1-xk1)-(m_master.x2-xk2)<=0)
    m, results = solve(m_master, 'gurobi')
    y_k = [m.y1(),m.y2(),m.y3()]
    if ub_temp<=UB:
        UB = ub_temp
    LB = m.alpha()
    print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
    print('UB:', UB, 'and LB:', LB)
    print()
    


Iteration1
optimal
NLP: | x1: 1.9999998371824865 | x2: 1.9999996377374851 | y1: 1.0 | y2: 1.0 | y3: 1.0 | Objective: 10.999997899680045 | Solver Status.: optimal
Master MILP: | y1: 1.0 | y2: -0.0 | y3: -0.0 | Objective: 1.0000014490499014 | Solver Status.: optimal
UB: 10.999997899680045 and LB: 1.0000014490499014

Iteration2
optimal
NLP: | x1: 2.0 | x2: 0.00045849332886224145 | y1: 1.0 | y2: 0.0 | y3: 0.0 | Objective: 5.0000002102161325 | Solver Status.: optimal
Master MILP: | y1: -0.0 | y2: 1.0 | y3: -0.0 | Objective: 1.5009167764415703 | Solver Status.: optimal
UB: 5.0000002102161325 and LB: 1.5009167764415703

Iteration3
optimal
NLP: | x1: 1.0 | x2: 1.0 | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 3.5 | Solver Status.: optimal
Master MILP: | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 3.5 | Solver Status.: optimal
UB: 3.5 and LB: 3.5



In [35]:
# Extended Cutting Planes
def master_ecp():
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    m.alpha = pyo.Var(within=pyo.Reals)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha

    m.alpha_cons = pyo.ConstraintList()
    m.g1_lins = pyo.ConstraintList()

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    return m

m = master_ecp()
y1 = 1
y2 = 1
y3 = 1

xk1 = 0
xk2 = 0
oj = float('-inf') 
oj1 = xk1**2+xk2**2+y1+1.5*y2+0.5*y3
while((xk1-2)**2 - xk2>0 or oj1>oj):
    m.alpha_cons.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
    m.g1_lins.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )
    m, results = solve(m, 'gurobi')
    print('Master MILP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
    #print('Objective Deviation',pyo.value(m.y1) + 1.5*pyo.value(m.y2) + 0.5*pyo.value(m.y3) + pyo.value(m.x1)**2 + pyo.value(m.x2)**2,'-',pyo.value(m.obj), '=', pyo.value(m.y1) + 1.5*pyo.value(m.y2) + 0.5*pyo.value(m.y3) + pyo.value(m.x1)**2 + pyo.value(m.x2)**2 - pyo.value(m.obj) ,'>',1e-4)
    print('Value of g1:',(m.x1()-2)**2 - m.x2())
    print()
    xk1 = pyo.value(m.x1)
    xk2 = pyo.value(m.x2)
    y1 = pyo.value(m.y1)
    y2 = pyo.value(m.y2)
    y3 = pyo.value(m.y3)
    oj = m.alpha()
    oj1 = pyo.value(m.y1) + 1.5*pyo.value(m.y2) + 0.5*pyo.value(m.y3) + pyo.value(m.x1)**2 + pyo.value(m.x2)**2
    print('Objective Deviation:',oj,'-',oj1)


Master MILP: | x1: 4.0 | x2: 4.0 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal
Value of g1: 0.0

Objective Deviation: 0.5 - 32.5
Master MILP: | x1: 1.6875 | x2: 1.6875 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal
Value of g1: -1.58984375

Objective Deviation: 0.5 - 6.1953125
Master MILP: | x1: 2.0 | x2: 0.0 | y1: 1.0 | y2: -0.0 | y3: -0.0 | Objective: 2.054687499999981 | Solver Status.: optimal
Value of g1: 0.0

Objective Deviation: 2.054687499999981 - 5.0
Master MILP: | x1: 1.0 | x2: 1.0 | y1: -0.0 | y2: 1.0 | y3: -0.0 | Objective: 2.554687499999972 | Solver Status.: optimal
Value of g1: 0.0

Objective Deviation: 2.554687499999972 - 3.5
Master MILP: | x1: 1.0 | x2: 1.0 | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 3.5 | Solver Status.: optimal
Value of g1: 0.0

Objective Deviation: 3.5 - 3.5


In [36]:
#Extended supporting hyperplane
def NLP_esh():
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.y2 = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.y3 = pyo.Var(within=pyo.Reals, bounds = (0,1))

    m.beta = pyo.Var(within=pyo.Reals)
    #m.alpha = pyo.Var(within=pyo.Reals)
    m.con_extra = ConstraintList()

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.beta

    #@m.Constraint()
    #def alpha_con(m):
     #   return m.beta >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + m.x1**2 + m.x2**2 -m.alpha
    #m.con_extra.add(m.alpha<=1+1.5+0.5)


    @m.Constraint()
    def g1_lin(m):
        return (m.x1-2)**2 - m.x2 <= m.beta 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    return m
NLP_model = NLP_esh()
NLP_model,results = solve(NLP_model, "gurobi")
x1_NLP = NLP_model.x1()
x2_NLP = NLP_model.x2()
y1_NLP = NLP_model.y1()
y2_NLP = NLP_model.y2()
y3_NLP = NLP_model.y3()
#alpha_NLP = NLP_model.alpha()
print('NLP: | x1:',x1_NLP,'| x2:',x2_NLP, '| y1:',y1_NLP, '| y2:',y2_NLP, '| y3:',y3_NLP,'| Objective:',pyo.value(NLP_model.beta))
def Line_search_esh(xk1,xk2,yk1,yk2,yk3):
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.y2 = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.y3 = pyo.Var(within=pyo.Reals, bounds = (0,1))

    #m.alpha = pyo.Var(within=pyo.Reals)
    m.beta = pyo.Var(within=pyo.Reals)
    m.lam = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.con_extra = ConstraintList()

    @m.Objective(sense=pyo.maximize)
    def obj(m):
        return m.beta
    m.con_extra.add(m.beta==0)
    m.con_extra.add(m.x1==m.lam*(xk1)+(1-m.lam)*(x1_NLP))
    m.con_extra.add(m.x2==m.lam*(xk2)+(1-m.lam)*(x2_NLP))
    m.con_extra.add(m.y1==m.lam*(yk1)+(1-m.lam)*(y1_NLP))
    m.con_extra.add(m.y2==m.lam*(yk2)+(1-m.lam)*(y2_NLP))
    m.con_extra.add(m.y3==m.lam*(yk3)+(1-m.lam)*(y3_NLP))
    #m.con_extra.add(m.alpha==m.lam*(alphak)+(1-m.lam)*(alpha_NLP))
    #m.con_extra.add(m.beta>=m.y1 + 1.5*m.y2 + 0.5*m.y3 + m.x1**2 + m.x2**2-m.alpha)
    m.con_extra.add(m.beta>=(m.x1-2)**2 - m.x2)
    m.con_extra.add(m.beta==0)
    #@m.Constraint()
    #def alpha_con(m):
     #   return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2)


    #@m.Constraint()
    #def g1_lin(m):
        #return (m.x1-2)**2 - m.x2 <= m.alpha 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0
    m, results = solve(m, 'gurobi')
    return m

def master_esh():
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.y2 = pyo.Var(within=pyo.Reals, bounds = (0,1))
    m.y3 = pyo.Var(within=pyo.Reals, bounds = (0,1))

    m.alpha = pyo.Var(within=pyo.Reals)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha

    m.con_extra = pyo.ConstraintList()
    xk1 = 0
    xk2 = 0
    yk1 = 1
    yk2 = 1
    yk3 = 1
    #alpha_k = 1+1.5+0.5
    m.con_extra.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
    m.con_extra.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )
    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0
    

    return m
m = master_esh()
yk1 = 1
yk2 = 1
yk3 = 1

xk1 = 0
xk2 = 0
oj =  float('-inf') 
oj1 = xk1**2+xk2**2+y1+1.5*y2+0.5*y3
eps = 0.0001
m.y1.domain = Binary
m.y2.domain = Binary
m.y3.domain = Binary
print("MILP Phase")
oj =  float('-inf') 
oj1 = xk1**2+xk2**2+yk1+1.5*yk2+0.5*yk3
eps = 0.0001
print(oj1>oj+eps)
while((xk1-2)**2 - xk2>0+eps or oj1>oj+eps):
    m, results = solve(m, 'gurobi')
    print('Master MILP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| alpha:',pyo.value(m.alpha),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
    #print('Objective Deviation',pyo.value(m.y1) + 1.5*pyo.value(m.y2) + 0.5*pyo.value(m.y3) + pyo.value(m.x1)**2 + pyo.value(m.x2)**2,'-',pyo.value(m.obj), '=', pyo.value(m.y1) + 1.5*pyo.value(m.y2) + 0.5*pyo.value(m.y3) + pyo.value(m.x1)**2 + pyo.value(m.x2)**2 - pyo.value(m.obj) ,'>',1e-4)
    oj = m.alpha()
    oj1 = pyo.value(m.y1) + 1.5*pyo.value(m.y2) + 0.5*pyo.value(m.y3) + pyo.value(m.x1)**2 + pyo.value(m.x2)**2
    print('Value of g1:',(m.x1()-2)**2 - m.x2())
    print('Value of g2:',oj1-oj)
    print(oj1)
    print(oj)
    xk1_MILP = pyo.value(m.x1)
    xk2_MILP = pyo.value(m.x2)
    yk1_MILP = pyo.value(m.y1)
    yk2_MILP = pyo.value(m.y2)
    yk3_MILP = pyo.value(m.y3)
    alphak_MILP = pyo.value(m.alpha)
    m_lin = Line_search_esh(xk1_MILP,xk2_MILP,yk1_MILP,yk2_MILP,yk3_MILP)
    
    xk1 = pyo.value(m_lin.x1)
    xk2 = pyo.value(m_lin.x2)
    yk1 = pyo.value(m_lin.y1)
    yk2 = pyo.value(m_lin.y2)
    yk3 = pyo.value(m_lin.y3)
    print('Line Search: | xk1:',xk1,'| xk2:',xk2, '| y1:',yk1, '| y2:',yk2, '| y3:',yk3)
    #m.con_extra.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1_LP**2 + xk2_LP**2 + 2*xk1_LP*(m.x1 - xk1_LP) + 2*xk2_LP*(m.x2 - xk2_LP))
    m.con_extra.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1_MILP**2 + xk2_MILP**2 + 2*xk1_MILP*(m.x1 - xk1_MILP) + 2*xk2_MILP*(m.x2 - xk2_MILP))
    if((xk1-2)**2 - xk2==0):
        m.con_extra.add(2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )
   

NLP: | x1: 2.0000000040362753 | x2: 4.0 | y1: 0.35438245265897833 | y2: 1.0 | y3: 1.0 | Objective: -4.0
MILP Phase
True
Master MILP: | x1: 4.0 | x2: 4.0 | y1: 0.0 | y2: 0.0 | y3: 1.0 | alpha: 0.5 | Objective: 0.5 | Solver Status.: optimal
Value of g1: 0.0
Value of g2: 32.0
32.5
0.5
Line Search: | xk1: 2.6290527470957743 | xk2: 4.0 | y1: 0.2429198254653897 | y2: 0.6854736278354929 | y3: 1.0
Master MILP: | x1: 1.6875 | x2: 1.6875 | y1: 0.0 | y2: 0.0 | y3: 1.0 | alpha: 0.5 | Objective: 0.5 | Solver Status.: optimal
Value of g1: -1.58984375
Value of g2: 5.6953125
6.1953125
0.5
Line Search: | xk1: 1.8443178278536951 | xk2: 2.847951911128853 | y1: 0.1778351543604937 | y2: 0.5018170426503148 | y3: 1.0
Master MILP: | x1: 2.0 | x2: 0.0 | y1: 1.0 | y2: -0.0 | y3: -0.0 | alpha: 2.054687499999981 | Objective: 2.054687499999981 | Solver Status.: optimal
Value of g1: 0.0
Value of g2: 2.945312500000019
5.0
2.054687499999981
Line Search: | xk1: 2.0000000040362753 | xk2: 0.0 | y1: 1.0 | y2: 0.0 | y3: 0